# GoogLeNet/InceptionNet

In [2]:
import tensorflow as tf
import numpy as np
import DL_utils as dl

In [3]:
def create_placeholders(n_H, n_W, n_C, n_y):
    '''
    Function to create placeholder for tensorflow session
    
    Args:
    n_H = height of the image
    n_W = width of image
    n_C = number of channels
    n_y = number of output features
    
    returns:
    X,Y
    '''
    X = tf.placeholder(tf.float32, shape = (None, n_H, n_W, n_C))
    Y = tf.placeholder(tf.float32, shape = (None, n_y))
    
    return X, Y

In [4]:
def IN_forward_propagation(X, parameters):
    '''
    forward propagation for the GoogLeNet architecture. This architecture uses the Inception Modules
    '''
    # 1st conv
    A1 = dl.conv_layer(X,parameters['W1'],parameters['b1'], [1,2,2,1], padding='SAME', name='1')
    # 1st pool
    P1 = dl.max_pool(A1, kernel=[1,3,3,1], strides=[1,2,2,1], padding = 'VALID', name='1')
    # 2nd conv
    A2 = dl.conv_layer(P1,parameters['W2'],parameters['b2'], [1,1,1,1], padding='SAME', name='2')
    # 2nd pool
    P2 = dl.max_pool(A2, kernel=[1,3,3,1], strides=[1,2,2,1], padding = 'VALID', name='2')
    # 1st Inception
    A3 = dl.inception_module(P2, parameters['inception3a'], name='3a')
    # 2nd Inception
    A4 = dl.inception_module(A3, parameters['incepiton3b'], name='3b')
    # 3rd pool
    P3 = dl.max_pool(A4, kernel=[1,3,3,1], strides=[1,2,2,1], padding = 'VALID', name='3')
    # 3rd Inception
    A5 = dl.inception_module(P3, parameters['inception4a'], name='4a')
    # 4th inception
    A6 = dl.inception_module(A5, parameters['inception4b'], name='4b')
    # 5th inception
    A7 = dl.inception_module(A6, parameters['inception4c'], name='4c')
    # 6th inception
    A8 = dl.inception_module(A7, parameters['inception4d'], name='4d')
    # 7th inception
    A9 = dl.inception_module(A8, parameters['inception4e'], name='4e')
    # 4th pool
    P4 = dl.max_pool(A9, kernel=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='4')
    # 8th inception
    A10 = dl.inception_module(A9, parameters['inception5a'], name='5a')
    # 9th inception
    A11 = dl.inception_module(A10, parameters['inception5b'], name='5b')
    # 5th pool
    P5 = dl.avg_pool(A11, kernel=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='4')
    
    # dropout normalization
    A_f = dl.dropout(P5, 0.4, name='1')
    
    F1 = dl.fc_layer(A_f, 1000, name='1')
    return F1

In [5]:
def compute_cost(F, Y):
    '''
    F = forward propagation result
    Y = actual labels
    '''
    with tf.name_scope('cost'):
        res = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=F)
        return res

In [ ]:
def model(xml_path, LOGDIR, MODELDIR, X_test=None, Y_test=None, learning_rate = 0.09, num_epochs = 100, 
          minibatch_size = 64, print_cost = True):
    
    # restting the default graph
    tf.reset_default_graph()
    
    # retrieve image shapes
    #(m, n_Htr, n_Wtr, n_Ctr) = X_train.shape
    #n_Y = Y_train.shape
    
    # global variables
    costs = []
    
    # randomizer
    seed = int(np.random.randint(1,100,1))
    
    # creating placeholders 
    X, Y = create_placeholders(448,448,3,1000)
    
    # initializing parameters
    parameters = initialize_weights(xml_path)
    
    # forward prop
    F1 = IN_forward_propagation(X, pretrain_parameters)
    
    # compute cost
    with tf.name_scope("cost"):
        cost = compute_cost(F1, Y)
    
    # select the appropriate the optimizer
    with tf.name_scope("train")
        train_step = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # initialize global variables
    init = tf.global_variables_initializer()
    
    # train the session
    with tf.Session() as sess:
        
        # run the initialization for the session
        sess.run(init)
        
        # initializing summaries and file writers
        merged_summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter(LOGDIR)
        writer.add_graph(sess.graph)
        saver = tf.train.Saver()
        
        # for loop for epoch/iterations
        for epoch in range(num_epochs):
            
            # maintain the cost through an epoch
            epoch_cost = 0
            
            # randomizer
            seed += 1
            
            # path to training folder
            PATH = './ImageNet_dataset/training_folder/'
            
            # set up the data
            h5_files = os.listdir(PATH)
            
            # for loop to iterate through the h5 files
            for file in h5_files:
                
                # DIAGNOSTIC print
                print("setting up" + file + "for training")
                
                # open the h5 file to form tensor
                with h5py.File(PATH+file, mode = 'r') as h5_file:
                    
                    # extract features and labels
                    X_train = np.asarray(h5_file['X_train'])
                    Y_train = np.asarray(h5_file['Y_train'])
                    
                    # number of examples
                    (m, n_Htr, n_Wtr, n_Ctr) = h5_file['X_train'].shape
                    
                    # ??? - REASON WHY
                    num_minibatches = int(m/minibatch_size)

                    # generate minibatches
                    minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

                    # iterate through the minibatches
                    for minibatch in minibatches:
                        
                        # procure minibatches
                        (minibatch_X, minibatch_Y) = minibatch
                        # optimize for cost, 
                        _ , minibatch_cost = sess.run([train_step, cost], feed_dict={X: minibatch_X, Y: tf.one_hot(minibatch_Y, 1000)})
                        # cumulative minibatch cost
                        epoch_cost += minibatch_cost/num_minibatches
                        
                    if epoch % 5 == 0:
                        s = sess.run(merged_summary, feed_dict = {X: X_train, Y: Y_train})
                        writer.add_summary(s, epoch)
                        save_path = saver.save(sess, MODELDIR)
            
            # Print the cost after every 5 epochs
            if print_cost == True and epoch % 1 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
                costs.append(epoch_cost)
                
    return pretrain_parameters